In [1]:
# !conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fnh3e5_f
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-fnh3e5_f
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print(clip.available_models())


['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [3]:

model, preprocess = clip.load("RN50", device=device)

task_description = ["A bookshelf placed against a wall"]

# Load and preprocess user-uploaded image
image = preprocess(Image.open("upload.jpeg")).unsqueeze(0).to(device)

# Convert text to CLIP embeddings
text_tokens = clip.tokenize(task_description).to(device)

# Extract features
with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text_tokens)

# Normalize features
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)

100%|███████████████████████████████████████| 244M/244M [00:35<00:00, 7.19MiB/s]


In [4]:
# Compute cosine similarity between text and image
similarity_score = (text_features @ image_features.T).item()
print(f"Similarity Score: {similarity_score:.4f}")


Similarity Score: 0.2280
